In [1]:
# third party
import sys
import matplotlib.pyplot as plt
import numpy as np
import sys
from pathlib import Path
import psyplot.project as psy
import pandas as pd
import xarray
from netCDF4 import Dataset,date2num
import metpy.calc as calc
from metpy.units import units
import datetime as dt
import pandas as pd
from iconarray.plot import formatoptions # import plotting formatoptions (for use with psyplot)
import iconarray as iconvis # import self-written modules from iconarray
from wrf import getvar, interp1d

# first party
sys.path.append('../utilities_tlezuo/')
from timefunctions import *
import varfunctions as vf
import locfunctions as lf

INFO:numexpr.utils:Note: NumExpr detected 36 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [2]:
## DECIDE UNIVERSAL PARAMS ##
# where to save
savepath = '/users/tlezuo/icon-vis/height_time_diagrams/data/'

In [3]:
## CROSSINN KOLSASS DATA ##
# from where to load
rs_path = '/users/tlezuo/data/2D/crossinn_rs/'
minlen = 1701

# decide which variables to calculate
pvars_list= [vf.VEL, vf.DIR, vf.T, vf.P, vf.ALT, vf.RH, vf.TD] 
cvars_list = [vf.U, vf.V,vf.TH,vf.QV]

# get model stuff for interpolation
loadpath = '/users/tlezuo/icon-vis/height_time_diagrams/data/'
loc=lf.kols
minlen_modlev = 73
htd_data_model = np.load(loadpath+'htd_ICON_'+loc.short+'.npy',allow_pickle='TRUE').item()
lev_model = htd_data_model['HFL_loc']
lev_model_cut = np.array(lev_model[0:minlen_modlev])

# create time lists
# time [str]
rstime_list = ['12_23','13_03','13_06','13_09','13_11','13_13','13_15','13_17','13_20','13_23']
# time [dt]
plotdates = [dt.datetime(2019,9,12,23,00),dt.datetime(2019,9,13,3,00),dt.datetime(2019,9,13,6,00),dt.datetime(2019,9,13,9,00),
dt.datetime(2019,9,13,11,00),dt.datetime(2019,9,13,13,00),dt.datetime(2019,9,13,15,00),dt.datetime(2019,9,13,17,00),dt.datetime(2019,9,13,20,00),dt.datetime(2019,9,13,23,00)]
# time [filename]
rsname_list = ['201909'+t+'UTC_L2.txt'for t in rstime_list]

# initialize dict
lidar_data_rs = {}
lidar_data_rs_interp = {}
# fill arrays to call them next
for pvar in pvars_list:
    lidar_data_rs[pvar.name] = np.zeros((len(rstime_list),minlen)) 
    lidar_data_rs_interp[pvar.name] = np.zeros((len(rstime_list),minlen_modlev)) 
    # lidar_data_rs_interp[pvar.name+'_diff'] = np.zeros((len(rstime_list),minlen_modlev))
for cvar in cvars_list:
    lidar_data_rs[cvar.name] =  np.zeros((len(rstime_list),minlen)) 
    lidar_data_rs_interp[cvar.name] = np.zeros((len(rstime_list),minlen_modlev)) 
    # lidar_data_rs_interp[cvar.name+'_diff'] = np.zeros((len(rstime_list),minlen_modlev))


# loop over sounding txt files
for counter,rstime in enumerate(rstime_list):
    # load filename
    rsname = rsname_list[counter]
    
    # read file
    rs_df = pd.read_fwf(rs_path+rsname,names=['time','P','T','RH','VEL','DIR','ALT','GEO','TD'],index_col=False)
    # drop first row
    rs_df.drop(index=rs_df.index[0], axis=0, inplace=True)

    # save each column in right dict location in dict
    for pvar in pvars_list:
            lidar_data_rs[pvar.name][counter] = rs_df[pvar.name][0:minlen]

## CALCULATIONS ##
# non interp
lidar_data_rs['TH'] =calc.potential_temperature(lidar_data_rs['P']* units.hPa,lidar_data_rs['T']* units.celsius)
lidar_data_rs['QV']  = calc.specific_humidity_from_dewpoint(lidar_data_rs['P']* units.hPa, lidar_data_rs['TD']* units.celsius)
lidar_data_rs['U'],lidar_data_rs['V'] = calc.wind_components(lidar_data_rs['VEL']* units('m/s'),lidar_data_rs['DIR']* units.deg)
lidar_data_rs['T'] = lidar_data_rs['T']+273.15

## INTERPOLATIONS ##
for counter,rstime in enumerate(rstime_list):
    # get levels of rs for interpolation
    lev_rs = np.array(lidar_data_rs['ALT'][counter][0:minlen]) # old and wrong: np.array(rs_df['ALT'][0:minlen])
    
    for pvar in pvars_list:
            sounding_rs = np.array(lidar_data_rs[pvar.name][counter])
            lidar_data_rs_interp[pvar.name][counter]  = interp1d(sounding_rs, lev_rs, np.transpose(lev_model_cut))
    for cvar in cvars_list:
            sounding_rs = np.array(lidar_data_rs[cvar.name][counter])
            lidar_data_rs_interp[cvar.name][counter]  = interp1d(sounding_rs, lev_rs, np.transpose(lev_model_cut))
# recalculate wind direction as it might be wrongly interp (360vs0-problem)
lidar_data_rs_interp['DIR'] = calc.wind_direction(lidar_data_rs_interp['U']* units('m/s'),lidar_data_rs_interp['V']* units('m/s'), convention='from')

## ADD DIFF VARIABLES ## 
# # (only for vertically interpolated data as they have same levels = length)
# for now in range(1,len(plotdates)):
#     # dates
#     pdate_now = plotdates[now]
#     pdate_before = plotdates[now-1]
#     delt=pdate_now-pdate_before
    
#     # compute difference for each variable
#     for pvar in pvars_list:
#         lidar_data_rs_interp[pvar.name+'_diff'][now]  = np.array(lidar_data_rs_interp[pvar.name][now,:]) - np.array(lidar_data_rs_interp[pvar.name][now-1,:])/(delt.seconds/3600)

#     for cvar in cvars_list:
#         lidar_data_rs_interp[cvar.name+'_diff'][now]  = np.array(lidar_data_rs_interp[cvar.name][now,:]) - np.array(lidar_data_rs_interp[cvar.name][now-1,:])/(delt.seconds/3600)

## SAVE LIDAR DATA ##
np.save(savepath+'htd_rs_Kolsass.npy', lidar_data_rs) 
np.save(savepath+'htd_rs_interp_Kolsass.npy', lidar_data_rs_interp) 

In [4]:
# MUN test
rs_path = '/users/tlezuo/data/2D/operational_rs/'
rsname = 'rs_mun_2019091212.txt'
rs_df = pd.read_csv(rs_path+rsname,delim_whitespace=True,names=['P','ALT','T','TD','RH','QV','DIR','VEL','TH','THE','THTV'],index_col=False)
rs_df

,P,ALT,T,TD,RH,QV,DIR,VEL,TH,THE,THTV
0,PRES,HGHT,TEMP,DWPT,RELH,MIXR,DRCT,SKNT,THTA,THTE,THTV
1,hPa,m,C,C,%,g/kg,deg,knot,K,K,K
2,1000.0,267,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,974.0,492,20.2,10.2,53,8.08,275,4,295.6,319.2,297.0
4,953.0,680,17.2,8.2,55,7.21,267,5,294.4,315.4,295.6
...,...,...,...,...,...,...,...,...,...,...,...
95,16.0,28195,-50.1,-84.2,1,0.02,250,10,726.9,727.2,727.0
96,15.6,28360,-48.3,-83.3,1,0.02,258,14,738.1,738.4,738.2
97,15.0,28618,-48.7,-83.3,1,0.03,270,19,745.1,745.4,745.1
98,14.2,28978,-49.3,-83.3,1,0.03,274,15,754.9,755.2,754.9


In [5]:
############################################################################################
## MUN SOUNDINGS ## 
# from where to load
rs_path = '/users/tlezuo/data/2D/operational_rs/'
minlen = 97 # minimal lenght of obs soundings

# decide which variables to calculate
pvars_list= [vf.VEL, vf.DIR, vf.T, vf.P, vf.ALT, vf.RH, vf.TD,vf.QV,vf.TH,] 
cvars_list = [vf.U, vf.V,]

## LOAD DATA ##
# get model stuff for interpolation
loadpath = '/users/tlezuo/icon-vis/height_time_diagrams/data/'
minlen_modlev = 73 # model level where to cut them
loc=lf.murs
htd_data_model = np.load(loadpath+'htd_ICON_'+loc.short+'.npy',allow_pickle='TRUE').item()
lev_model = htd_data_model['HFL_loc']
lev_model_cut = np.array(lev_model[0:minlen_modlev])

# create time lists
# time [str]
rstime_list = ['1212','1300','1312','1400']
# time [dt]
plotdates = [dt.datetime(2019,9,12,12,00),dt.datetime(2019,9,13,0,00),dt.datetime(2019,9,13,12,00),dt.datetime(2019,9,14,0,00)]
# time [filename]
rsname_list = ['rs_mun_201909'+t+'.txt'for t in rstime_list]

# initialize dict
lidar_data_rs = {}
lidar_data_rs_interp = {}
# fill arrays to call them next
for pvar in pvars_list:
    lidar_data_rs[pvar.name] = np.zeros((len(rstime_list),minlen)) 
    lidar_data_rs_interp[pvar.name] = np.zeros((len(rstime_list),minlen_modlev)) 
for cvar in cvars_list:
    lidar_data_rs[cvar.name] =  np.zeros((len(rstime_list),minlen)) 
    lidar_data_rs_interp[cvar.name] = np.zeros((len(rstime_list),minlen_modlev)) 

# loop over sounding txt files
for counter,rstime in enumerate(rstime_list):
    # load filename
    rsname = rsname_list[counter]
    
    # read file
    rs_df = pd.read_csv(rs_path+rsname,delim_whitespace=True,names=['P','ALT','T','TD','RH','QV','DIR','VEL','TH','THE','THTV'],index_col=False)
    # drop first and second row
    rs_df.drop([0,1], axis=0, inplace=True)
    

    # save each column in right dict location in dict
    for pvar in pvars_list:
            lidar_data_rs[pvar.name][counter] = rs_df[pvar.name][0:minlen]

## CALCULATIONS ##
# non interp
lidar_data_rs['VEL'] = lidar_data_rs['VEL']/1.94384 # knots to m/s
lidar_data_rs['U'],lidar_data_rs['V'] = calc.wind_components(lidar_data_rs['VEL']* units('m/s'),lidar_data_rs['DIR']* units.deg)
lidar_data_rs['T'] = lidar_data_rs['T']+273.15
lidar_data_rs['TD'] = lidar_data_rs['TD']+273.15

## INTERPOLATIONS ##
for counter,rstime in enumerate(rstime_list):
    # get levels of rs for interpolation
    lev_rs = np.array(lidar_data_rs['ALT'][counter][0:minlen])
    for pvar in pvars_list:
            sounding_rs = np.array(lidar_data_rs[pvar.name][counter])
            lidar_data_rs_interp[pvar.name][counter]  = interp1d(sounding_rs, lev_rs, np.transpose(lev_model_cut))
    for cvar in cvars_list:
            sounding_rs = np.array(lidar_data_rs[cvar.name][counter])
            lidar_data_rs_interp[cvar.name][counter]  = interp1d(sounding_rs, lev_rs, np.transpose(lev_model_cut))
# recalculate wind direction as it might be wrongly interp (360vs0-problem)
lidar_data_rs_interp['DIR'] = calc.wind_direction(lidar_data_rs_interp['U']* units('m/s'),lidar_data_rs_interp['V']* units('m/s'), convention='from')

## SAVE HTD DATA ##
np.save(savepath+'htd_rs_Munich.npy', lidar_data_rs) 
np.save(savepath+'htd_rs_interp_Munich.npy', lidar_data_rs_interp) 

In [6]:
# Innsbruck test
rs_path = '/users/tlezuo/data/2D/operational_rs/'
rsname = 'rs_ifl_2019091302.csv'
rs_df = pd.read_csv(rs_path+rsname,names=['i','t','ALT','P','T','TD','DIR','VEL'],index_col=False)
rs_df

,i,t,ALT,P,T,TD,DIR,VEL
0,NaN,timestamp,altitude,press,temp,dewp_temp,wind_dir,wind_vel
1,0.0,2019-09-13 02:00:00,282.0,1000.0,NaN,NaN,NaN,NaN
2,1.0,2019-09-13 02:00:00,578.0,965.5,10.3,9.54,270.0,0.5
3,2.0,2019-09-13 02:00:00,646.0,957.7,12.54,10.94,276.0,1.8
4,3.0,2019-09-13 02:00:00,883.0,931.0,12.82,9.96,241.0,1.8
...,...,...,...,...,...,...,...,...
93,92.0,2019-09-13 02:00:00,24562.0,28.0,-59.1,-89.48,65.0,5.5
94,93.0,2019-09-13 02:00:00,24647.0,27.6,-58.72,-89.68,75.0,5.0
95,94.0,2019-09-13 02:00:00,25110.0,25.6,-56.99,-89.01,249.0,0.2
96,95.0,2019-09-13 02:00:00,25401.0,24.5,-55.87,-88.25,331.0,2.2


In [7]:
############################################################################################
## INNSBRUCK SOUNDINGS ## 
# from where to load
rs_path = '/users/tlezuo/data/2D/operational_rs/'
minlen = 96 # minimal lenght of obs soundings

# decide which variables to calculate
pvars_list= [vf.VEL, vf.DIR, vf.T, vf.P, vf.ALT, vf.TD,] 
cvars_list = [vf.U, vf.V, vf.RH, vf.QV,vf.TH]

## LOAD DATA ##
# get model stuff for interpolation
loadpath = '/users/tlezuo/icon-vis/height_time_diagrams/data/'
minlen_modlev = 73 # model level where to cut them
loc=lf.ifl
htd_data_model = np.load(loadpath+'htd_ICON_'+loc.short+'.npy',allow_pickle='TRUE').item()
lev_model = htd_data_model['HFL_loc']
lev_model_cut = np.array(lev_model[0:minlen_modlev])

# create time lists
# time [str]
rstime_list = ['1302',]
# time [dt]
plotdates = [dt.datetime(2019,9,13,2,00),]
# time [filename]
rsname_list = ['rs_ifl_201909'+t+'.csv'for t in rstime_list]

# initialize dict
lidar_data_rs = {}
lidar_data_rs_interp = {}
# fill arrays to call them next
for pvar in pvars_list:
    lidar_data_rs[pvar.name] = np.zeros((len(rstime_list),minlen)) 
    lidar_data_rs_interp[pvar.name] = np.zeros((len(rstime_list),minlen_modlev)) # model levels they reach
for cvar in cvars_list:
    lidar_data_rs[cvar.name] =  np.zeros((len(rstime_list),minlen)) # 81 levels for halflevel-variables as W
    lidar_data_rs_interp[cvar.name] = np.zeros((len(rstime_list),minlen_modlev)) # 81 levels for halflevel-variables as W


# loop over sounding txt files
for counter,rstime in enumerate(rstime_list):
    # load filename
    rsname = rsname_list[counter]
    
    # read file
    rs_df = pd.read_csv(rs_path+rsname,names=['i','t','ALT','P','T','TD','DIR','VEL'],index_col=False)
    # drop first and second row
    rs_df.drop([0,1], axis=0, inplace=True)
    # drop first and second column index and timestamp
    rs_df.drop(['i','t'], axis=1, inplace=True)

    # save each column in right dict location in dict
    for pvar in pvars_list:
            lidar_data_rs[pvar.name][counter] = rs_df[pvar.name][0:minlen]

## CALCULATIONS ##
# non interp
lidar_data_rs['U'],lidar_data_rs['V'] = calc.wind_components(lidar_data_rs['VEL']* units('m/s'),lidar_data_rs['DIR']* units.deg)
lidar_data_rs['TH'] = calc.potential_temperature(lidar_data_rs['P']* units.hPa,lidar_data_rs['T']* units.celsius)
lidar_data_rs['RH'] = calc.relative_humidity_from_dewpoint(lidar_data_rs['T']* units.celsius, lidar_data_rs['TD']* units.celsius)
lidar_data_rs['QV']  = calc.specific_humidity_from_dewpoint(lidar_data_rs['P']* units.hPa, lidar_data_rs['TD']* units.celsius)
lidar_data_rs['T'] = lidar_data_rs['T']+273.15
lidar_data_rs['TD'] = lidar_data_rs['TD']+273.15


## INTERPOLATION TO MODEL LEVELS ##
for counter,rstime in enumerate(rstime_list):
    # get levels of rs for interpolation
    lev_rs = np.array(rs_df['ALT'][0:minlen])
    for pvar in pvars_list:
            sounding_rs = np.array(lidar_data_rs[pvar.name][counter])
            lidar_data_rs_interp[pvar.name][counter]  = interp1d(sounding_rs, lev_rs, np.transpose(lev_model_cut))
    for cvar in cvars_list:
            sounding_rs = np.array(lidar_data_rs[cvar.name][counter])
            lidar_data_rs_interp[cvar.name][counter]  = interp1d(sounding_rs, lev_rs, np.transpose(lev_model_cut))
# recalculate wind direction as it might be wrongly interp (360vs0-problem)
lidar_data_rs_interp['DIR'] = calc.wind_direction(lidar_data_rs_interp['U']* units('m/s'),lidar_data_rs_interp['V']* units('m/s'), convention='from')

## INTERPOLATION TO HOUR TIMESTEPS ##




## SAVE LIDAR DATA ##
np.save(savepath+'htd_rs_Innsbruck.npy', lidar_data_rs) 
np.save(savepath+'htd_rs_interp_Innsbruck.npy', lidar_data_rs_interp)